<a href="https://colab.research.google.com/github/michaeledge27/CSCI290/blob/main/notebooks/premierLeagueProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [181]:
url = "https://github.com/michaeledge27/CSCI290/raw/main/data/matches.csv"
df = pd.read_csv(url)
df.head()

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,0,2020-09-21,20:15 (21:15),Premier League,Matchweek 2,Mon,Away,W,3,1,...,Match Report,NaN,13,8,21.1,2,1,1,2024,Manchester City
1,2,2020-09-27,16:30 (17:30),Premier League,Matchweek 3,Sun,Home,L,2,5,...,Match Report,NaN,16,5,19.8,1,0,0,2024,Manchester City
2,4,2020-10-03,17:30 (18:30),Premier League,Matchweek 4,Sat,Away,D,1,1,...,Match Report,NaN,23,1,18.2,1,0,0,2024,Manchester City
3,5,2020-10-17,17:30 (18:30),Premier League,Matchweek 5,Sat,Home,W,1,0,...,Match Report,NaN,13,5,17.7,0,0,0,2024,Manchester City
4,7,2020-10-24,12:30 (13:30),Premier League,Matchweek 6,Sat,Away,D,1,1,...,Match Report,NaN,14,7,20.9,1,0,0,2024,Manchester City


In [182]:
df2 = (df['result']=='W') * 1
df2.head()

,result
0,1
1,0
2,0
3,1
4,0


In [183]:
from sklearn.model_selection import train_test_split

In [184]:
train_set, test_set = train_test_split(df, test_size = 0.2, shuffle = True)

In [185]:
def Attribute_selection_method(dataFrame, target, measure):
  features = dataFrame.columns[dataFrame.columns != target]
  splittingMeasure = dataFrame.columns[df.columns[1] if df.columns[1] != target else df.columns[2]]
  if measure == 'entropy':
    # calculate max entropy
    for feature in features:
      entropiesList = []
      counts = np.bincount(feature)
      countProb = counts/len(features)
      entropyValue = -np.sum(countProb * np.log2(countProb))
      entropiesList.append(entropyValue)
      return features[entropiesList.index(min(entropiesList))]


  elif measure == 'gini':
    # calculate max entropy
    for feature in features:
      giniList = []
      counts = np.bincount(feature)
      countProb = counts/len(features)
      giniValue = 1 - np.sum(np.square(countProb))
      giniList.append(giniValue)
      return features[giniList.index(max(giniList))]

  else:
    return None


In [186]:
train_set.columns

Index(['Unnamed: 0', 'date', 'time', 'comp', 'round', 'day', 'venue', 'result',
       'gf', 'ga', 'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain',
       'formation', 'referee', 'match report', 'notes', 'sh', 'sot', 'dist',
       'fk', 'pk', 'pkatt', 'season', 'team'],
      dtype='object')

In [187]:
train_set = train_set[['round', 'venue', 'xga', 'poss', 'sot', 'fk', 'pk', 'result']]

In [188]:
test_set = test_set[['round', 'venue', 'xga', 'poss', 'sot', 'fk', 'pk', 'result']]

In [189]:
X_train = train_set[['round', 'venue', 'xga', 'poss', 'sot', 'fk', 'pk']]
y_train = train_set[['result']]

In [190]:
X_test = test_set[['round', 'venue', 'xga', 'poss', 'sot', 'fk', 'pk']]
y_test = test_set[['result']]

In [191]:
from sklearn.tree import DecisionTreeClassifier

In [192]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree

In [193]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3830 entries, 2753 to 3271
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   round   3830 non-null   object 
 1   venue   3830 non-null   object 
 2   xga     3830 non-null   float64
 3   poss    3830 non-null   int64  
 4   sot     3830 non-null   int64  
 5   fk      3830 non-null   int64  
 6   pk      3830 non-null   int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 239.4+ KB


In [194]:
num_attributes = X_train.select_dtypes(include = ['int64', 'float64']).columns
cat_attributes = X_train.select_dtypes(include = ['object']).columns

In [195]:
trf = [ ('num', StandardScaler(), num_attributes),
       ('cat', OneHotEncoder( handle_unknown='ignore'), cat_attributes) ]
col_transform = ColumnTransformer( transformers = trf )

In [196]:
pipeline = Pipeline( steps = [('pre', col_transform),
 ('clf', DecisionTreeClassifier(max_depth=3))])

In [197]:
cross_val_score( pipeline, X_train, y_train, cv=5, scoring='accuracy')

array([0.55091384, 0.58877285, 0.57571802, 0.56788512, 0.58877285])

In [198]:
model = pipeline.fit( X_train, y_train )
model

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['xga', 'poss', 'sot', 'fk', 'pk'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['round', 'venue'], dtype='object'))])),
                ('clf', DecisionTreeClassifier(max_depth=3))])

In [199]:
predictions = pipeline.predict( X_test )

In [200]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [201]:
cm = confusion_matrix( y_test, predictions )
cm

array([[  3, 129,  79],
       [  9, 316,  49],
       [  2, 101, 270]])

In [202]:
df2 = (df['result']=='W') * 1
df2.head()

,result
0,1
1,0
2,0
3,1
4,0


In [203]:
def Attribute_selection_method_2(dataFrame, target, measure):


SyntaxError: incomplete input (<ipython-input-203-9fc9949dfe44>, line 2)